<a href="https://colab.research.google.com/github/GGpark1/User-Log-data-analytics/blob/main/refactoring_CF_electronics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas.io import gbq

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
import warnings
warnings.simplefilter("ignore")

## Collaborative Filtering 추천 시스템

### 추천 모델 구현을 위한 데이터 전처리
- Big-query를 활용하여 로그 데이터와 item 데이터 import
- 한 제품을 6개 이상 구매한 데이터 제외
- 구매 정보가 없는 데이터(Null)를 0으로 대체

In [3]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'electronics%' AND event_type = 'purchase'
GROUP BY user_id, product_id
HAVING purchase_count <= 5
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df_limit = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [4]:
sql = """SELECT product_id, brand, category_code FROM `empirical-weft-350709.CP2_ggpark.log_data`
WHERE event_type = 'purchase' AND category_code LIKE 'electronics%'
GROUP BY product_id, brand, category_code"""
project_id = 'empirical-weft-350709'

df_product = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [5]:
df_limit.sort_values('purchase_count')

,user_id,product_id,purchase_count
146438,536389259,1005031,1
195242,555969459,4804409,1
195243,565029018,4804409,1
195244,558455237,4803720,1
195245,534723329,4802248,1
...,...,...,...
1609,539104297,1005126,5
1608,538085722,1005115,5
1607,539688983,1005115,5
1605,539625884,1003304,5


In [6]:
df_ratings_limit = df_limit
mean = df_ratings_limit['purchase_count'].mean()

In [7]:
df_fin_limit = pd.pivot_table(df_limit, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')
df_fin_limit

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
303160429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340041246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401021311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403013066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410333972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566263549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566265448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566270060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_final_limit = df_fin_limit.fillna(0)

In [10]:
df_base_line_limit = df_fin_limit.fillna(mean)

In [9]:
df_final_limit['total'] = df_final_limit.sum(axis=1)
df_filtered_limit = df_final_limit.loc[df_final_limit['total'] > 1]
df_filtered_limit.drop(columns='total', inplace = True)

df_filtered_limit

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415873351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
427391662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
437371552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446080337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566240992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566243643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566249028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Collaborative Filtering 구현
- SVD 행렬 분해 방식으로 구현
    - 차원 수가 많은 데이터에 유리한 알고리즘
    - 희소 문제에 유리한 알고리즘

In [11]:
# Matrix 계산을 위해 pandas를 numpy로 변환
matrix_limit = df_filtered_limit.values

# 유저의 평균 구매횟수
user_ratings_mean_limit = np.mean(matrix_limit, axis = 1)

# 각 유저의 구매횟수에서 평균 구매횟수를 뺌으로써 평균에서 벗어난 특이값을 남김
user_ratings_mean_reshape_limit = user_ratings_mean_limit.reshape(-1, 1)
matrix_user_mean_limit = matrix_limit - user_ratings_mean_reshape_limit

In [12]:
# 행렬 분해
U_limit, sigma_limit, Vt_limit = svds(matrix_user_mean_limit, k = 1500)

In [13]:
# 0이 포함된 대칭행렬로 전환
sigma_limit = np.diag(sigma_limit)

In [14]:
# 분해한 행렬에서 특이값을 추출한 후 다시 하나로 복구함
dot_matrix_limit = np.dot(U_limit, sigma_limit)
df_svd_predicted_purchase_limit = np.dot(dot_matrix_limit, Vt_limit)

# 앞서 뺐던 유저의 평균 구매 횟수를 다시 더함
df_svd_predicted_purchase_limit = df_svd_predicted_purchase_limit + user_ratings_mean_reshape_limit

In [15]:
# 유저별로 제품에 대한 구매 확률을 예측한 테이블

df_svd_preds_limit = pd.DataFrame(df_svd_predicted_purchase_limit, columns = df_filtered_limit.columns, index = df_filtered_limit.index)
df_svd_preds_limit.head()

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400236,51600003,51600004,51600005,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,-0.000027,-0.000078,-0.000011,-1.844444e-06,0.000014,2.510372e-06,0.000002,4.767164e-07,0.000003,-0.000004,...,-0.000004,-0.000006,-2.386311e-05,-0.000004,-3.828594e-06,-7.944489e-07,-2.638824e-06,-0.000004,-8.626415e-07,9.891402e-07
415873351,-0.000378,0.000647,0.000497,1.151548e-05,-0.000052,3.712169e-06,-0.000013,1.332890e-05,0.000045,-0.000003,...,-0.000003,0.000002,2.434774e-06,-0.000003,-6.139606e-06,-3.157914e-06,3.263817e-06,-0.000003,-6.882603e-07,8.031662e-07
427391662,0.000207,-0.000032,-0.000016,-1.274986e-07,0.000032,6.272417e-07,0.000010,-9.823506e-06,-0.000025,-0.000004,...,-0.000004,-0.000004,-7.789485e-07,-0.000004,-1.450870e-06,-3.726095e-06,-3.032085e-06,-0.000004,-2.248923e-06,1.085517e-06
437371552,0.000209,-0.000076,-0.000016,1.067950e-05,0.000029,-2.384822e-05,0.000025,-7.908708e-06,-0.000020,-0.000003,...,-0.000003,0.000002,-2.819100e-06,-0.000003,-7.455053e-07,3.082122e-06,1.331747e-07,-0.000003,1.292177e-07,8.549083e-07
446080337,-0.000211,-0.000009,-0.000008,9.520907e-06,-0.000023,4.537092e-06,-0.000008,1.314003e-05,0.000025,-0.000006,...,-0.000006,-0.000005,-8.343319e-07,-0.000006,-8.750478e-06,-2.583966e-06,-7.172958e-06,-0.000006,-4.591142e-06,7.860599e-07


In [16]:
# 유저 Id를 입력하면 추천 아이템이 출력되는 함수

def recommend_items(df_svd_preds, user_id, ori_items_df, ori_ratings_df, num_recommendations=5):
    
    user_row_number = user_id 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 데이터 정렬 -> 구매 확률이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 item 데이터를 합친다. 
    user_history = user_data.merge(ori_items_df, on = 'product_id').sort_values(['purchase_count'], ascending=False)
    
    # 유저가 이미 구매한 item을 제외하고 데이터 추출
    recommendations = ori_items_df[~ori_items_df['product_id'].isin(user_history['product_id'])]

    # 정렬된 추천 list와 item 정보를 merge하여 하나의 테이블로 만듦 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'product_id')
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [17]:
already_rated, predictions = recommend_items(df_svd_preds_limit, 512363712, df_product, df_ratings_limit, 10)

In [18]:
predictions

,product_id,brand,category_code,Predictions
2267,1801306,haier,electronics.video.tv,0.008426
561,1005088,tp-link,electronics.smartphone,0.003292
874,1003114,sony,electronics.smartphone,0.003059
3092,21405783,casio,electronics.clocks,0.002847
1413,1802069,philips,electronics.video.tv,0.002780
2151,1801632,philips,electronics.video.tv,0.002712
1084,1802123,artel,electronics.video.tv,0.002537
4251,22400001,ritmix,electronics.audio.microphone,0.002040
2202,14100634,denn,electronics.audio.acoustic,0.001149
443,8800516,texet,electronics.telephone,0.001116


### 평가지표
- RMSE : 실제 데이터의 잠재 요인을 얼마나 잘 추출했는지 원본 데이터와의 오차로 비교함
- Base Line
    - 평균 구매 횟수로 채운 테이블과 원본 데이터와의 MSE를 산출하여 모든 사용자가 평균만큼은 구매할 것으로 예측하고 오차를 계산함
    - Score : 0.88
- Truncated SVD 
    - SVD로 예측된 테이블과 원본 데이터와의 MSE를 산출하여 추출된 잠재 요인만큼 상품을 구매할 것으로 예측하고 오차를 계산함
    - Score : 0.15


In [19]:
def evaluate(sparse_matrix, pred_matrix):
        idx, jdx = sparse_matrix.to_numpy().nonzero()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(sparse_matrix.iloc[i, j])
            preds.append(pred_matrix.iloc[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [20]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_base_line_limit)

0.8815595103052039

In [21]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_svd_preds_limit)

0.15935074508688535